In [116]:
import numpy as np
from collections.abc import Callable
import functools

In [117]:
input = open('data/input.txt', 'r').read().split('\n')

In [118]:
N = int(input[0])
hard = np.array(list(map(int, input[1].split())))
time = np.array(list(map(float, input[2].split())))
M = int(input[3])
devs = np.array([list(map(float,i.split())) for i in input[4:]])

In [119]:
i = np.arange(N)
j = np.arange(M)

i, j = np.meshgrid(i, j, indexing='ij')
combination = np.stack([i, j], axis=-1)

In [120]:
rng = np.random.default_rng(seed=22)

In [121]:
def create_subject() -> np.ndarray:
    idx = rng.integers(0, M, size=N)
    return combination[np.arange(N), idx]

In [122]:
def create_population(k: int) -> np.ndarray:
    idx = rng.integers(0, M, size=(k, N))
    rows = np.arange(N)[None, :]
    return combination[rows, idx]

In [123]:
def fitness_numpy(subject: np.ndarray) -> np.float64:    
    tasks = subject[:, 0]
    devs_idx = subject[:, 1]
    task_times = time[tasks] * devs[devs_idx, hard[tasks] - 1]
    dev_time = np.bincount(devs_idx, weights=task_times)

    return dev_time.max()

In [124]:
def fitness_numpy_population(population: np.ndarray) -> np.ndarray:
    k = population.shape[0]
    tasks = population[:, :, 0]
    devs_idx = population[:, :, 1]
    task_times = time[tasks] * devs[devs_idx, hard[tasks] - 1]
    total_time = np.zeros((k, M), dtype=np.float64)
    np.add.at(total_time, (np.arange(k)[:, None], devs_idx), task_times)

    return total_time.max(axis=1)

In [125]:
def selection_elitism_and_roulette(fitness: np.ndarray, n: int, elitism_count: int = 5) -> np.ndarray:
    n = n-elitism_count
    return np.concatenate((fitness.argsort()[:elitism_count], rng.choice(fitness.shape[0], size=n, replace=False)))

In [126]:
def two_point_crossover(population: np.ndarray, n: int, p: float = 0.7) -> np.ndarray:
    k = population.shape[0]

    parent_left = rng.choice(k, size=n)
    parent_right = (parent_left + rng.integers(1, k, size=n)) % k

    f_left  = fitness_numpy_population(population[parent_left])
    f_right = fitness_numpy_population(population[parent_right])

    p = np.where(f_left < f_right, p, 1-p)

    mask = rng.random(size=(n, N)) < p[:, None]

    children = population[parent_left].copy()

    children[:, :, 1] = np.where(
        mask,
        population[parent_right, :, 1],
        children[:, :, 1]
    )
    
    return children

In [127]:
def mutate(
    children: np.ndarray,
    p_subject_mutation: float = 0.1,
    p_gen_mutation: float = 0.1
) -> np.ndarray:

    k = children.shape[0]

    mutated = children.copy()

    mask_subject = rng.random(size=k) < p_subject_mutation
    mask_gen = rng.random(size=(k, N)) < p_gen_mutation

    full_mask = mask_subject[:, None] & mask_gen

    random_devs = rng.integers(0, M, size=(k, N))

    mutated[:, :, 1] = np.where(
        full_mask,
        random_devs,
        mutated[:, :, 1]
    )

    return mutated

In [128]:
def train(
    create_population: Callable[[], np.ndarray],
    fitness: Callable[[np.ndarray], np.ndarray],
    selection: Callable[[np.ndarray], np.ndarray],
    crossover: Callable[[np.ndarray, int], np.ndarray],
    mutation: Callable[[np.ndarray], np.ndarray],
    n_best: int,
    n_iter: int
) -> np.ndarray:
    population = create_population()

    for _ in range(n_iter):
        f = fitness(population)
        selected = population[selection(f)]
        children = crossover(selected, population.shape[0] - selected.shape[0])
        children = mutation(children)
        population = np.concatenate([selected, children], axis=0)
    
    return population[np.argsort(fitness(population))[:n_best]]

In [129]:
solutions = train(
        create_population=functools.partial(create_population, 100),
        fitness=fitness_numpy_population,
        selection=functools.partial(selection_elitism_and_roulette, n=25),
        crossover=two_point_crossover,
        mutation=functools.partial(mutate, p_subject_mutation=0.001, p_gen_mutation=0.5),
        n_best=1,
        n_iter=5000
    )

In [130]:
solutions = solutions.reshape(N,2)

In [131]:
fitness_numpy(solutions)

np.float64(604.44)

In [132]:
solutions = solutions+1

In [133]:
res = " ".join(map(str, solutions[:, 1]))

In [134]:
open('data/output.txt','w').write(res)

2102

by P.S.Y.